In [1]:
import string
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.stem import PorterStemmer, SnowballStemmer, LancasterStemmer
from sklearn.decomposition import TruncatedSVD, NMF
from gensim import corpora,models,matutils

In [2]:
df=pd.read_csv("/Users/jaszeng/Desktop/Metis/NLP/Corona_NLP_train.csv",encoding='latin-1')

In [3]:
df.iloc[7484,].OriginalTweet

'Our partners at @NetElixir are using their knowledge and tools to track changes in online shopping behavior as the situation surrounding the novel coronavirus develops. Stay up to date: https://t.co/hcael4jXeL'

In [40]:
df.info

<bound method DataFrame.info of        UserName  ScreenName                      Location     TweetAt  \
0          3799       48751                        London  16-03-2020   
1          3800       48752                            UK  16-03-2020   
2          3801       48753                     Vagabonds  16-03-2020   
3          3802       48754                           NaN  16-03-2020   
4          3803       48755                           NaN  16-03-2020   
...         ...         ...                           ...         ...   
41152     44951       89903  Wellington City, New Zealand  14-04-2020   
41153     44952       89904                           NaN  14-04-2020   
41154     44953       89905                           NaN  14-04-2020   
41155     44954       89906                           NaN  14-04-2020   
41156     44955       89907  i love you so much || he/him  14-04-2020   

                                           OriginalTweet           Sentiment  
0      @MeNy

In [5]:
df.TweetAt.max()

'31-03-2020'

In [6]:
df.TweetAt.min()

'01-04-2020'

In [7]:
docs=df.OriginalTweet

In [8]:
vec = CountVectorizer(stop_words='english',max_df=0.8)
doc_term = vec.fit_transform(docs)
doc_term.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [2, 0, 0, ..., 0, 0, 0]])

In [9]:
pd.unique(doc_term.toarray().reshape(-1))

array([ 0,  1,  3,  2,  4,  6,  5,  7, 11,  8,  9])

In [10]:
doc_term.shape

(41157, 80120)

In [11]:
vec.vocabulary_

{'menyrbie': 45492,
 'phil_gahan': 53896,
 'chrisitv': 15863,
 'https': 34361,
 'ifz9fan2pa': 35276,
 'xx6ghgfzcc': 77924,
 'i2nlzdxno8': 34819,
 'advice': 6758,
 'talk': 67868,
 'neighbours': 48618,
 'family': 26666,
 'exchange': 25973,
 'phone': 53943,
 'numbers': 50087,
 'create': 19070,
 'contact': 17568,
 'list': 42517,
 'schools': 62108,
 'employer': 24888,
 'chemist': 15568,
 'gp': 31130,
 'set': 62851,
 'online': 51357,
 'shopping': 63454,
 'accounts': 6362,
 'poss': 54912,
 'adequate': 6603,
 'supplies': 67015,
 'regular': 58757,
 'meds': 45321,
 'order': 51669,
 'coronavirus': 18070,
 'australia': 9557,
 'woolworths': 76527,
 'elderly': 24583,
 'disabled': 21939,
 'dedicated': 20849,
 'hours': 34128,
 'amid': 7964,
 'covid': 18565,
 '19': 913,
 'outbreak': 51952,
 'binca9vp8p': 11639,
 'food': 28150,
 'stock': 66180,
 'don': 22673,
 'panic': 52713,
 'need': 48554,
 'stay': 65803,
 'calm': 14134,
 'safe': 61289,
 'covid19france': 18605,
 'covid_19': 18722,
 'covid19': 18572,
 

In [14]:
dictt=pd.DataFrame.from_dict(vec.vocabulary_,orient='index',columns=['count'])

In [15]:
dictt.head()

,count
menyrbie,45492
phil_gahan,53896
chrisitv,15863
https,34361
ifz9fan2pa,35276


In [16]:
dictt.sort_values(by='count',ascending=False).head(20)

,count
ã¼retime,80119
ã¼cretsiz,80118
ã¼berweist,80117
â²,80116
zã¼rich,80115
zã,80114
zâ,80113
zzzz,80112
zzz,80111
zzxwzxhc2v,80110


In [17]:
stemmer = PorterStemmer()
analyzer = CountVectorizer(stop_words='english',max_df=0.7,min_df=5).build_analyzer()
def stemmed_words(doc):
    return (stemmer.stem(w) for w in analyzer(doc))

In [18]:
stem_vec = CountVectorizer(stop_words='english',max_df=0.7,min_df=5,analyzer=stemmed_words)

In [19]:
doc_term = stem_vec.fit_transform(docs)

In [20]:
doc_term.shape

(41157, 7917)

In [21]:
stemmer = PorterStemmer()
analyzer = TfidfVectorizer(stop_words='english',max_df=0.7,min_df=5).build_analyzer()
stem_vec_ti=TfidfVectorizer(stop_words='english',max_df=0.7,min_df=5,analyzer=stemmed_words)

In [22]:
doc_term_ti = stem_vec_ti.fit_transform(docs)

In [23]:
doc_term_ti.shape

(41157, 7917)

## Applying LSA, NMP and LDA to Doc-term Matrix

### 1. LSA

In [29]:
lsa=TruncatedSVD(5)
lsa.fit(doc_term_ti)

TruncatedSVD(n_components=5)

In [30]:
lsa.components_

array([[ 3.46154033e-03,  1.60461805e-02,  4.11315712e-04, ...,
         2.78063206e-04,  1.59958320e-04,  3.93666294e-04],
       [ 5.91006342e-04,  3.88407536e-03,  6.72714024e-05, ...,
         1.00559482e-04,  8.53053870e-05,  3.21020066e-04],
       [ 1.71726366e-03, -3.08696137e-04, -3.78829358e-04, ...,
        -5.46933225e-04, -5.67559692e-05,  5.58372299e-04],
       [-6.69459602e-04, -3.94573205e-03, -5.65616350e-04, ...,
        -9.99963723e-04, -6.42322802e-05,  1.41738085e-04],
       [-4.02144344e-04, -7.20311320e-04,  2.07401598e-04, ...,
        -3.72856171e-04, -1.26423738e-04, -7.99305897e-05]])

In [32]:
for i in range(5):
    topic = pd.Series(data=lsa.components_[i], 
                index=stem_vec_ti.get_feature_names()).sort_values(ascending=False)
    print('topic',(i+1),':\n',topic.head(10),"\n")

topic 1 :
 http           0.393739
coronaviru     0.285554
19             0.248876
covid          0.248038
store          0.209492
groceri        0.191348
food           0.167507
supermarket    0.156726
price          0.154528
shop           0.151341
dtype: float64 

topic 2 :
 covid     0.309435
19        0.305900
price     0.256990
consum    0.228565
http      0.179491
oil       0.109670
impact    0.090815
demand    0.084135
market    0.070809
pandem    0.057453
dtype: float64 

topic 3 :
 coronaviru    0.410904
http          0.365962
sanit         0.272798
hand          0.246064
covid19       0.238202
toiletpap     0.198158
toilet        0.131310
paper         0.127672
price         0.075494
mask          0.054536
dtype: float64 

topic 4 :
 store      0.329211
groceri    0.309891
http       0.234220
consum     0.163578
covid      0.135345
19         0.122896
retail     0.089250
pandem     0.058694
worker     0.057073
employe    0.056055
dtype: float64 

topic 5 :
 shop           0.

### 2. NMF

In [33]:
nmf=NMF(5)
nmf.fit(doc_term_ti)

/opt/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "


NMF(n_components=5)

In [34]:
nmf.components_

array([[1.36961586e-02, 4.68821002e-02, 1.04248760e-03, ...,
        0.00000000e+00, 8.26048761e-04, 1.45559522e-03],
       [2.69001038e-03, 1.04779905e-02, 0.00000000e+00, ...,
        0.00000000e+00, 6.32769291e-05, 0.00000000e+00],
       [1.02883961e-02, 3.62990839e-02, 0.00000000e+00, ...,
        0.00000000e+00, 1.09445522e-04, 2.93800508e-03],
       [4.84118056e-03, 3.86984917e-02, 2.89865543e-03, ...,
        4.17500540e-03, 6.12337524e-04, 0.00000000e+00],
       [4.13193905e-03, 3.22140677e-02, 8.96280714e-04, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [36]:
for i in range(5):
    topic = pd.Series(data=nmf.components_[i], 
                index=stem_vec_ti.get_feature_names()).sort_values(ascending=False)
    print('topic',(i+1),':\n',topic.head(10),"\n")

topic 1 :
 19        2.716245
covid     2.706329
price     1.694938
consum    1.384917
http      1.326864
oil       0.645973
pandem    0.593175
impact    0.564391
demand    0.472612
market    0.470735
dtype: float64 

topic 2 :
 store       2.357966
groceri     2.212937
worker      0.828823
work        0.465572
employe     0.411187
thank       0.324511
line        0.305253
covid_19    0.294121
retail      0.290710
go          0.283629
dtype: float64 

topic 3 :
 http          2.165071
coronaviru    1.993414
covid19       1.138734
sanit         0.976821
hand          0.895632
toiletpap     0.749211
toilet        0.490162
paper         0.485758
use           0.277381
mask          0.267245
dtype: float64 

topic 4 :
 food           1.927148
supermarket    1.226219
buy            1.162889
panic          1.104949
peopl          1.050589
need           0.765444
stock          0.661051
amp            0.532079
suppli         0.461995
shelv          0.432718
dtype: float64 

topic 5 :
 shop   

### 3. LDA

In [38]:
corpus=matutils.Sparse2Corpus(doc_term.transpose())
id2word=dict((v,k) for k,v in stem_vec.vocabulary_.items())
lda=models.LdaModel(corpus=corpus,num_topics=5,id2word=id2word)

In [39]:
lda.print_topics()

[(0,
  '0.051*"http" + 0.038*"19" + 0.038*"covid" + 0.031*"consum" + 0.031*"price" + 0.016*"coronaviru" + 0.013*"amp" + 0.011*"market" + 0.009*"new" + 0.008*"chang"'),
 (1,
  '0.036*"http" + 0.031*"shop" + 0.028*"19" + 0.027*"covid" + 0.026*"onlin" + 0.018*"supermarket" + 0.015*"coronaviru" + 0.011*"mask" + 0.010*"store" + 0.009*"amp"'),
 (2,
  '0.047*"coronaviru" + 0.034*"http" + 0.022*"sanit" + 0.019*"hand" + 0.018*"covid19" + 0.016*"supermarket" + 0.014*"food" + 0.013*"peopl" + 0.011*"toiletpap" + 0.010*"buy"'),
 (3,
  '0.052*"http" + 0.031*"price" + 0.029*"19" + 0.028*"covid" + 0.019*"consum" + 0.019*"food" + 0.017*"coronaviru" + 0.015*"demand" + 0.013*"pandem" + 0.012*"oil"'),
 (4,
  '0.039*"groceri" + 0.039*"store" + 0.022*"worker" + 0.019*"coronaviru" + 0.019*"http" + 0.012*"supermarket" + 0.012*"peopl" + 0.011*"work" + 0.009*"amp" + 0.008*"socialdistanc"')]